In [1]:
import time
import numpy as np
import pandas as pd

from pprint import pprint
from datetime import datetime, date, timedelta

# Import API key
from config import API_KEY, POSTGRES_USERNAME, POSTGRES_PASSWORD, POSTGRES_ADDRESS, POSTGRES_PORT, POSTGRES_DBNAME

In [2]:
from sqlalchemy import create_engine

# A long string that contains the necessary Postgres login information
postgres_str = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}"

# Create the connection
cnx = create_engine(postgres_str)

# SETUP ONLY

### Call our Spotify APIs

In [3]:
try:
    myTestdf = pd.read_sql_query("SELECT * FROM active_user LIMIT 5", cnx)
    
except Error as e:
    print(f"<message>./n/n  Error is:  {e}")
finally:
    print("Done.")
# end try

Done.


In [4]:
myTestdf.head()

,id,first_name,last_name,username
0,6,Fraser,Korneev,fstappard5
1,7,Demott,Rapson,lhambling6
2,15,Sadella,Jaram,wheinerte
3,17,Hewitt,Trammel,droughsedgeg
4,19,Ted,Knowlys,ydudeniei
